# Build P9 (ATPase) protein tree

In [1]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import pandas as pd
import os

In [2]:
def extract_faa(gb_file, name):
    i = 1
    
    faa_records = []
    
    for record in SeqIO.parse(gb_file, 'genbank'):
        for feature in record.features:
            if feature.type == 'CDS':
                aa_seq = feature.qualifiers['translation'][0]
                aa_record = SeqRecord(Seq(aa_seq), id=f'{name}_orf{i}', name='', description='')
                faa_records.append(aa_record)
                i += 1
    
    return faa_records

def read_tblout(tblout):
    df = pd.read_csv(tblout,
                 delim_whitespace=True,
                 comment='#',
                 header=None)
    
    # parse and label column
    df = df[df.columns[:5]]
    df.columns = ['target', '-', 'query', '-',  'full_eval']
    
    
    df['acc'] = df['target'].str.rsplit('_', 1, expand=True)[0]
    df = df[['acc', 'target', 'query', 'full_eval']]
    
    df = df[df['full_eval'] < 0.01]
    
    # select best hit for annotation
    df = df.sort_values(by=['acc', 'target', 'full_eval'], ascending=[True, True, False])
    df = df.drop_duplicates(subset=['target'], keep='first')
    
    return df

## Get P9 sequences

In [3]:
# list with aa sequences for P9 proteins
p9_list = []

# result will be saved here
p9_all_faa = '../data/tree/P9/P9.faa'

### This study's

In [4]:
# get selection of alphatectiviruses
sspp = ['PRDobsidian', 'PRDpistachio', 'PRDaquamarine',
        'PRDcanary', 'PRDsepia', 'PRDwine',
        'PRDavocado', 'PRDamber', 'PRDforest',
        'PRDcobalt', 'PRDindigo']


# append aminoacid sequences corresponding to those genomes
for record in SeqIO.parse('../data/models/faa/IX.faa', 'fasta'):
    isolate_name = record.id.split('_IX')[0]
    if isolate_name in sspp:
        record.id = isolate_name
        record.name = ''
        record.description = ''
        p9_list.append(record)

#### NCBI

In [5]:
# run hmmer with ATPase model on ncbi tectiviruses

#!hmmsearch \
#    --tblout '../data/tecti_genomes/NCBI/hmmer/p9.tbl' \
#    '../data/models/hmm/IX.2.hmm' \
#    '../data/tecti_genomes/NCBI/hmmer/all_ncbi_orfs.faa' \
#    > /dev/null

In [6]:
# add p9 hits to list
df_ncbi_hits = read_tblout('../data/tecti_genomes/NCBI/hmmer/p9.tbl')

for record in SeqIO.parse('../data/tecti_genomes/NCBI/hmmer/all_ncbi_orfs.faa', 'fasta'):
    if record.id in df_ncbi_hits[df_ncbi_hits['query'] == 'IX.2']['target'].to_list():
        record.name = ''
        record.description = ''
        p9_list.append(record)

/tmp/ipykernel_31346/3433487581.py:27: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df['acc'] = df['target'].str.rsplit('_', 1, expand=True)[0]


In [7]:
# add PRD1 and PR4
for record in SeqIO.parse('../data/tecti_genomes/NCBI/gb/ref_p9.faa', 'fasta'):
    p9_list.append(record)

#### JGI

In [8]:
# run hmmer with ATPase model on jgi imgvr tectiviruses

#!hmmsearch \
#    --tblout '../data/tecti_genomes/JGI_IMGVR/hmmer/p9.tbl' \
#    '../data/models/hmm/IX.2.hmm' \
#    '../data/tecti_genomes/JGI_IMGVR/hmmer/all_jgi_orfs.faa' \
#    > /dev/null

In [9]:
# add p9 hits to list
df_jgi_hits = read_tblout('../data/tecti_genomes/JGI_IMGVR/hmmer/p9.tbl')

for record in SeqIO.parse('../data/tecti_genomes/JGI_IMGVR/hmmer/all_jgi_orfs.faa', 'fasta'):
    if record.id in df_jgi_hits[df_jgi_hits['query'] == 'IX.2']['target'].to_list():
        record.name = ''
        record.description = ''
        p9_list.append(record)

/tmp/ipykernel_31346/3433487581.py:27: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df['acc'] = df['target'].str.rsplit('_', 1, expand=True)[0]


#### Yutin (2018)

In [10]:
# run hmmer with ATPase model on Yutin, et al. tectiviruses

#!hmmsearch \
#    --tblout '../data/tecti_genomes/Yutin/hmmer/p9.tbl' \
#    '../data/models/hmm/IX.2.hmm' \
#    '../data/tecti_genomes/Yutin/hmmer/all_yutin_orfs.faa' \
#    > /dev/null

In [11]:
# add p9 hits to list
df_yutin_hits = read_tblout('../data/tecti_genomes/Yutin/hmmer/p9.tbl')

for record in SeqIO.parse('../data/tecti_genomes/Yutin/hmmer/all_yutin_orfs.faa', 'fasta'):
    if record.id in df_yutin_hits[df_yutin_hits['query'] == 'IX.2']['target'].to_list():
        record.name = ''
        record.description = ''
        p9_list.append(record)

/tmp/ipykernel_31346/3433487581.py:27: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df['acc'] = df['target'].str.rsplit('_', 1, expand=True)[0]


In [12]:
# Save result
#SeqIO.write(p9_list, p9_all_faa, 'fasta')

## Explore results

### NCBI

In [13]:
# read hits table
df_ncbi_hits = read_tblout('../data/tecti_genomes/NCBI/hmmer/p9.tbl')

# read metadata table
df_ncbi_metadata = pd.read_csv('../data/tecti_genomes/NCBI/tectivirus_metadata.tsv', sep='\t')
df_ncbi_metadata = df_ncbi_metadata[df_ncbi_metadata['genus'] != '-']
df_ncbi_metadata = df_ncbi_metadata[df_ncbi_metadata['genus'] != 'Alphatectivirus']

pivot_ncbi_hits = df_ncbi_hits.pivot(index='acc', columns='query', values='full_eval').reset_index().fillna('-')
df_ncbi_metadata.merge(pivot_ncbi_hits, on='acc', how='left')

/tmp/ipykernel_31346/3433487581.py:27: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df['acc'] = df['target'].str.rsplit('_', 1, expand=True)[0]


,acc,genus,species_name,isolate,len,host_class,host_sp,IX.2
0,NC_011523.1,Betatectivirus,Betatectivirus AP50,Bacillus phage AP50,14398,Bacilli,Bacillus anthracis,2.500000e-55
1,NC_005258.1,Betatectivirus,Betatectivirus Bam35,Bacillus phage Bam35c,14935,Bacilli,Bacillus thuringiensis,3.100000e-58
2,NC_006945.1,Betatectivirus,Betatectivirus GIL16,Bacillus phage GIL16c,14844,Bacilli,Bacillus thuringiensis,9.600000e-57
3,MZ089978.1,Betatectivirus,Betatectivirus sato,Bacillus phage Sato,14852,Bacilli,Bacillus cereus,1.500000e-56
4,MZ089979.1,Betatectivirus,Betatectivirus sole,Bacillus phage Sole,14444,Bacilli,Bacillus cereus VD166,6.900000e-57
5,NC_022094.1,Betatectivirus,Betatectivirus Wip1,Bacillus phage Wip1,14319,Bacilli,Bacillus anthracis,3.900000e-55
6,NC_055059.1,Deltatectivirus,Deltatectivirus forthebois,Streptomyces phage Forthebois,18251,Actinomycetia,Streptomyces scabiei,2.800000e-36
7,NC_055060.1,Deltatectivirus,Deltatectivirus wheeheim,Streptomyces phage WheeHeim,18266,Actinomycetia,Streptomyces scabiei,9.300000e-36
8,NC_055061.1,Epsilontectivirus,Epsilontectivirus toil,Rhodococcus phage Toil,17253,Actinomycetia,Rhodococcus opacus,1.900000e-65
9,NC_042083.1,Gammatectivirus,Gammatectivirus GC1,Gluconobacter phage GC1,16523,Alphaproteobacteria,Gluconobacter cerinus,6.200000e-74


### JGI

In [14]:
df_jgi_hits = read_tblout('../data/tecti_genomes/JGI_IMGVR/hmmer/p9.tbl')

df_jgi_metadata = pd.read_csv('../data/tecti_genomes/JGI_IMGVR/JGI_metadata.tsv', sep='\t')

pivot_jgi_hits = df_jgi_hits.pivot(index='acc', columns='query', values='full_eval').reset_index().fillna('-')
df_jgi_metadata.merge(pivot_jgi_hits, on='acc', how='left')

/tmp/ipykernel_31346/3433487581.py:27: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df['acc'] = df['target'].str.rsplit('_', 1, expand=True)[0]


,acc,img_taxon_id,sample_name,len,loc,IX.2
0,Ga0500005_002616,3300050116,Peatland microbial communities from Stordalen ...,19730,sweden,4.000000e-48
1,Ga0500006_002682,3300050117,Peatland microbial communities from Stordalen ...,20094,sweden,4.000000e-48
2,Ga0500006_012856,3300050117,Peatland microbial communities from Stordalen ...,5558,sweden,1.700000e-45
3,Ga0500009_001431,3300050120,Peatland microbial communities from Stordalen ...,17036,sweden,7.200000e-59
4,Ga0500011_004634,3300050122,Peatland microbial communities from Stordalen ...,4396,sweden,3.400000e-62
5,Ga0500012_004241,3300050123,Peatland microbial communities from Stordalen ...,4422,sweden,3.300000e-62
6,Ga0500013_005094,3300050124,Peatland microbial communities from Stordalen ...,4816,sweden,3.300000e-62
7,Ga0500017_0003996,3300050128,Peatland microbial communities from Stordalen ...,11142,sweden,4.000000e-48
8,Ga0500018_001608,3300050129,Peatland microbial communities from Stordalen ...,18742,sweden,4.000000e-48
9,Ga0500022_002121,3300050132,Peatland microbial communities from Stordalen ...,18125,sweden,9.800000e-41


## Yutin

In [15]:
df_yutin_hits = read_tblout('../data/tecti_genomes/Yutin/hmmer/p9.tbl')

# explore results
df_yutin_metadata = pd.read_csv('../data/tecti_genomes/Yutin/yutin_metadata.tsv', sep='\t')

pivot_yutin_hits = df_yutin_hits.pivot(index='acc', columns='query', values='full_eval').reset_index().fillna('-')
df_yutin_metadata.merge(pivot_yutin_hits, on='acc', how='left')

/tmp/ipykernel_31346/3433487581.py:27: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df['acc'] = df['target'].str.rsplit('_', 1, expand=True)[0]


,acc,environment,len,IX.2
0,LNFM01013825.1,Activated carbon metagenome,10153,4.000000e-60
1,FRDC01003407.1,freshwater metagenome,17409,3.200000e-25
2,LNFM01009513.1,Activated carbon metagenome,14103,1.900000e-32
3,JRYJ01001167.1,Activated sludge metagenome,17872,5.600000e-29


## Build tree

In [16]:
# align resulting sequences to model
#!hmmalign --trim '../data/models/hmm/IX.2.hmm' '../data/tree/P9/P9.faa' | esl-reformat --gapsym='-' afa - > '../data/tree/P9/P9.afa'

In [17]:
# give them nice readable names for phylip format
name_dict = dict(zip(df_ncbi_metadata['acc'], df_ncbi_metadata['species_name'].str.replace('tectivirus', '').str.replace(' ', '-')))
name_dict.update(zip(df_jgi_metadata['acc'], df_jgi_metadata['loc'].str[:2] + df_jgi_metadata['acc'].str.split('_', expand=True)[1].str.lstrip('0')))
name_dict.update(zip(df_yutin_metadata['acc'], df_yutin_metadata['acc'].str.split('.', expand=True)[0]))

renamed = []
for record in SeqIO.parse('../data/tree/P9/P9.afa', 'fasta'):
    name = record.id.split('_orf')[0]
    if name in name_dict.keys():
        new_name = name_dict[name]
    else:
        new_name = name
        
    record.id = new_name
    
    renamed.append(record)
    
    
#SeqIO.write(renamed, '../data/tree/P9/P9.phy', 'phylip')

In [18]:
aln_phy = '../data/tree/P9/P9.phy'
phy_cmd = 'phyml '\
            '-d aa '\
            '-m LG ' \
            '-b -4 '\
            '-v 0.0 '\
            '-c 4 '\
            '-a e '\
            '-f e '\
            '--no_memory_check '\
            f'-i {aln_phy}'

In [19]:
#!phy_cmd

In [20]:
print(phy_cmd)

phyml -d aa -m LG -b -4 -v 0.0 -c 4 -a e -f e --no_memory_check -i ../data/tree/P9/P9.phy
